# Preparation of GWR - Data

## Libraries and Settings

In [2]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

c:\Users\dimit\Documents\applied_data_science\week_04\spatial_data_analysis\05_Python_GWR_Data


## Download GWR data

Source: https://public.madd.bfs.admin.ch

In [3]:
# Download latest file for the Canton of Zurich
url = "https://public.madd.bfs.admin.ch/buildings_zh.geojson"
response = requests.get(url)

# Ensure response is valid
if response.status_code == 200:
    # Open file in write mode and write the response content
    with open('buildings_zh.geojson', 'wb') as file:
        file.write(response.content)
else:
    print(f"Failed to download file, status code: {response.status_code}")

## Import latest GWR Data

In [4]:
# Load json file
with open('buildings_zh.geojson') as f:
    data = json.load(f)

# Flatten nested json data
df_orig = pd.json_normalize(data, record_path=['features'])
df_orig

# Remove prefix
df_orig.columns = df_orig.columns.str.replace('properties.', '')
df_orig.columns = df_orig.columns.str.replace('geometry.', '')

# Create copy
df = df_orig.copy()
df.head()

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,coordinates
0,Feature,1,1004,1040,1271,2,Affoltern am Albis,ZH,Point,"[2676523, 1235843]"
1,Feature,10,1004,1020,1110,2,Affoltern am Albis,ZH,Point,"[2676440.038, 1235973.538]"
2,Feature,100,1004,1020,1110,2,Affoltern am Albis,ZH,Point,"[2676765.706, 1236101.658]"
3,Feature,1000,1007,1030,1110,2,Affoltern am Albis,ZH,Point,"[2676644.881, 1236812.863]"
4,Feature,10000,1004,1030,1121,291,Andelfingen,ZH,Point,"[2693391.65, 1272255.267]"


## Separate Swiss LV95 coordinates

In [5]:
# Separate coordinates
df['x_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['x_coord'] 
df['y_coords'] = pd.DataFrame(df['coordinates'].tolist(), columns=['x_coord', 'y_coord'])['y_coord']

# Remove column 'coordinates'
df = df.drop(['coordinates'], axis=1)
df

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,x_coords,y_coords
0,Feature,1,1004,1040,1271,2,Affoltern am Albis,ZH,Point,2676523.000,1235843.000
1,Feature,10,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676440.038,1235973.538
2,Feature,100,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676765.706,1236101.658
3,Feature,1000,1007,1030,1110,2,Affoltern am Albis,ZH,Point,2676644.881,1236812.863
4,Feature,10000,1004,1030,1121,291,Andelfingen,ZH,Point,2693391.650,1272255.267
...,...,...,...,...,...,...,...,...,...,...,...
393859,Feature,99995,1004,1040,0,198,Uster,ZH,Point,2696792.976,1245085.637
393860,Feature,99996,1004,1040,0,198,Uster,ZH,Point,2696718.474,1245039.536
393861,Feature,99997,1004,1040,0,198,Uster,ZH,Point,2696722.040,1245043.472
393862,Feature,99998,1004,1040,0,198,Uster,ZH,Point,2696727.666,1245050.586


## Create WGS84 coordinates

In [12]:
# Create geodataframe and calculate latitude and longitude
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x_coords'], df['y_coords']), crs="EPSG:2056") 

# Convert the Swiss LV95 coordinates to lat & lon
gdf = gdf.to_crs(epsg=4326)

# Get Latitude and Longitude
df['latitude'] = gdf['geometry'].y
df['longitude'] = gdf['geometry'].x

# Show data
df

,type,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,canton,type,x_coords,y_coords,latitude,longitude
0,Feature,1,1004,1040,1271,2,Affoltern am Albis,ZH,Point,2676523.000,1235843.000,47.269056,8.449859
1,Feature,10,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676440.038,1235973.538,47.270239,8.448785
2,Feature,100,1004,1020,1110,2,Affoltern am Albis,ZH,Point,2676765.706,1236101.658,47.271354,8.453110
3,Feature,1000,1007,1030,1110,2,Affoltern am Albis,ZH,Point,2676644.881,1236812.863,47.277764,8.451635
4,Feature,10000,1004,1030,1121,291,Andelfingen,ZH,Point,2693391.650,1272255.267,47.594339,8.680355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
393859,Feature,99995,1004,1040,0,198,Uster,ZH,Point,2696792.976,1245085.637,47.349516,8.719673
393860,Feature,99996,1004,1040,0,198,Uster,ZH,Point,2696718.474,1245039.536,47.349113,8.718677
393861,Feature,99997,1004,1040,0,198,Uster,ZH,Point,2696722.040,1245043.472,47.349147,8.718725
393862,Feature,99998,1004,1040,0,198,Uster,ZH,Point,2696727.666,1245050.586,47.349211,8.718801


## Plot subset of buildings

In [18]:
# Subset
df_sub = df.loc[df['municipalityName'] == 'Neftenbach'].sample(100).dropna()
df_sub

# Create the map
m = folium.Map(location=[df_sub['latitude'].mean(), df_sub['longitude'].mean()], zoom_start=15)

# Add points to the map
for idx, row in df_sub.iterrows():
    folium.Marker(location=([row['latitude'], 
                            row['longitude']]),
                popup=row['buildingCategory']).add_to(m) # Add popup with building category

# Display the map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [8]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 11
Datetime: 2024-06-17 16:40:24
Python Version: 3.12.1
-----------------------------------
